In [1]:
using Revise

In [5]:
using RigidBodyDynamics
using RigidBodyDynamics: Bounds
using StaticArrays: SVector
using Gurobi
using Plots; gr()

Plots.GRBackend()

In [3]:
import LCPSim
import LearningMPC

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/LCPSim.ji for module LCPSim.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [71]:
world = RigidBody{Float64}("world")
mechanism = Mechanism(world; gravity=SVector(0, 0, -9.81))

frame = CartesianFrame3D("base")
inertia = SpatialInertia(frame, 1 * eye(3), zeros(3), 1.0)
base = RigidBody(inertia)
base_x = Joint("base_x", Prismatic([1., 0, 0]))
position_bounds(base_x) .= Bounds(-100., 100)
velocity_bounds(base_x) .= Bounds(-1000., 1000)
effort_bounds(base_x) .= Bounds(-1000., 1000)
attach!(mechanism, world, base, base_x)

xstar = MechanismState{Float64}(mechanism)
x0 = MechanismState{Float64}(mechanism)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=1,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, TimeLimit=120, MIPGap=1e-4, FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

environment = LCPSim.Environment{Float64}(Dict())

Q = diagm([1., 1])
R = diagm([0.1])
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, mpc_params.Δt)

mpc_controller = LearningMPC.MPCController(mechanism, 
    environment, mpc_params, lqrsol, 
    [lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}()

mpc_controller.callback = sample_sink

Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::MPCSampleSink) (generic function with 1 method)

In [72]:
set_configuration!(x0, [1.0])
results = LCPSim.simulate(x0, 
    mpc_controller, environment, mpc_params.Δt, 100, mpc_params.lcp_solver);
set_configuration!(x0, [1.0])
results_lqr = LCPSim.simulate(x0, 
    lqrsol, environment, mpc_params.Δt, 100, mpc_params.lcp_solver);

plt = plot([r.state.configuration[1] for r in results])
plot!(plt, [r.state.configuration[1] for r in results_lqr], label="lqr")

plt2 = plot([r.input[1] for r in results])
plot!(plt2, [r.input[1] for r in results_lqr], label="lqr")

plot(plt, plt2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 lqr 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 lqr

In [78]:
A = vcat([s.state' for s in sample_sink.samples]...)
b = [s.uJ[1] for s in sample_sink.samples]

100-element Array{Float64,1}:
 -3.05992  
 -2.40923  
 -1.8735   
 -1.43327  
 -1.07231  
 -0.777154 
 -0.536556 
 -0.341176 
 -0.183239 
 -0.0562766
  0.0450889
  0.12533  
  0.188162 
  ⋮        
  0.0186338
  0.0177792
  0.0169637
  0.0161857
  0.0154433
  0.014735 
  0.0140592
  0.0134144
  0.0127991
  0.012212 
  0.0116519
  0.0111175

In [79]:
A \ b

2-element Array{Float64,1}:
 -3.05992
 -4.09998

In [73]:
[s.uJ for s in sample_sink.samples]

100-element Array{Array{Float64,2},1}:
 [-3.05992 -3.05992 -4.09998]  
 [-2.40923 -3.05992 -4.09998]  
 [-1.8735 -3.05992 -4.09998]   
 [-1.43327 -3.05992 -4.09998]  
 [-1.07231 -3.05992 -4.09998]  
 [-0.777154 -3.05992 -4.09998] 
 [-0.536556 -3.05992 -4.09998] 
 [-0.341176 -3.05992 -4.09998] 
 [-0.183239 -3.05992 -4.09998] 
 [-0.0562766 -3.05992 -4.09998]
 [0.0450889 -3.05992 -4.09998] 
 [0.12533 -3.05992 -4.09998]   
 [0.188162 -3.05992 -4.09998]  
 ⋮                             
 [0.0186338 -3.05992 -4.09998] 
 [0.0177792 -3.05992 -4.09998] 
 [0.0169637 -3.05992 -4.09998] 
 [0.0161857 -3.05992 -4.09998] 
 [0.0154433 -3.05992 -4.09998] 
 [0.014735 -3.05992 -4.09998]  
 [0.0140592 -3.05992 -4.09998] 
 [0.0134144 -3.05992 -4.09998] 
 [0.0127991 -3.05992 -4.09998] 
 [0.012212 -3.05992 -4.09998]  
 [0.0116519 -3.05992 -4.09998] 
 [0.0111175 -3.05992 -4.09998] 

In [70]:
lqrsol.K

1×2 Array{Float64,2}:
 2.85872  3.72689

In [37]:
[r.input for r in results]

100-element Array{Array{Float64,1},1}:
 [-3.05992]  
 [-2.40923]  
 [-1.8735]   
 [-1.43327]  
 [-1.07231]  
 [-0.777154] 
 [-0.536556] 
 [-0.341176] 
 [-0.183239] 
 [-0.0562766]
 [0.0450889] 
 [0.12533]   
 [0.188162]  
 ⋮           
 [0.0186338] 
 [0.0177792] 
 [0.0169637] 
 [0.0161857] 
 [0.0154433] 
 [0.014735]  
 [0.0140592] 
 [0.0134144] 
 [0.0127991] 
 [0.012212]  
 [0.0116519] 
 [0.0111175] 

In [40]:
for s in sample_sink.samples
    @show (s.state, s.uJ, -lqrsol.K * s.state)
end

(s.state, s.uJ, -(lqrsol.K) * s.state) = ([1.0, 0.0], [-3.05992 -3.05992 -4.09998], [-2.85872])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.99235, -0.152996], [-2.40923 -3.05992 -4.09998], [-2.26665])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.978677, -0.273458], [-1.8735 -3.05992 -4.09998], [-1.77862])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.960321, -0.367133], [-1.43327 -3.05992 -4.09998], [-1.37702])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.938381, -0.438796], [-1.07231 -3.05992 -4.09998], [-1.04722])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.91376, -0.492412], [-0.777154 -3.05992 -4.09998], [-0.777019])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.887197, -0.53127], [-0.536556 -3.05992 -4.09998], [-0.556263])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.859292, -0.558097], [-0.341176 -3.05992 -4.09998], [-0.376507])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.830534, -0.575156], [-0.183239 -3.05992 -4.09998], [-0.23072])
(s.state, s.uJ, -(lqrsol.K) * s.state) = ([0.

In [38]:
sample_sink.samples

100-element Array{LearningMPC.Sample{Float64},1}:
 LearningMPC.Sample{Float64}([1.0, 0.0], [-3.05992 -3.05992 -4.09998], [25.9012], LearningMPC.MIPResults
  solvetime_s: Float64 0.00315499
  objective_value: Float64 25.8961
  objective_bound: Float64 25.8961
)                       
 LearningMPC.Sample{Float64}([0.99235, -0.152996], [-2.40923 -3.05992 -4.09998], [23.8376], LearningMPC.MIPResults
  solvetime_s: Float64 0.00293708
  objective_value: Float64 23.835
  objective_bound: Float64 23.835
)               
 LearningMPC.Sample{Float64}([0.978677, -0.273458], [-1.8735 -3.05992 -4.09998], [22.1421], LearningMPC.MIPResults
  solvetime_s: Float64 0.00258398
  objective_value: Float64 22.141
  objective_bound: Float64 22.141
)               
 LearningMPC.Sample{Float64}([0.960321, -0.367133], [-1.43327 -3.05992 -4.09998], [20.6721], LearningMPC.MIPResults
  solvetime_s: Float64 0.00572801
  objective_value: Float64 20.6717
  objective_bound: Float64 20.6717
)            
 LearningMPC.S

In [27]:
for i in 2:length(results)
    input = results[i].input
    lqr = lqrsol(results[i - 1].state)
    @show norm(input .- lqr)
end

norm(input .- lqr) = 0.1396048984499738
norm(input .- lqr) = 0.09246555159475767
norm(input .- lqr) = 0.0542823821727052
norm(input .- lqr) = 0.02351147613344029
norm(input .- lqr) = 0.0011312820320945605
norm(input .- lqr) = 0.02071364450602675
norm(input .- lqr) = 0.036123186330701884
norm(input .- lqr) = 0.04809745177008695
norm(input .- lqr) = 0.05724906840931121
norm(input .- lqr) = 0.06408666847409972
norm(input .- lqr) = 0.06903231684290745
norm(input .- lqr) = 0.07243602855688769
norm(input .- lqr) = 0.07458786142092722
norm(input .- lqr) = 0.07572798829520444
norm(input .- lqr) = 0.07605508618576451
norm(input .- lqr) = 0.07573332301563962
norm(input .- lqr) = 0.07489817609771854
norm(input .- lqr) = 0.07366127731335403
norm(input .- lqr) = 0.07211444743678164
norm(input .- lqr) = 0.07033305498478604
norm(input .- lqr) = 0.06837881235603716
norm(input .- lqr) = 0.0663021030348585
norm(input .- lqr) = 0.06414391935644348
norm(input .- lqr) = 0.06193747299517827
norm(input .- lq